In [86]:
from importlib import reload

import task1
reload(task1)
from task1 import *

import task2
reload(task2)
from task2 import *

path1="data/title.akas.tsv.gz"
path2="data/title.basics.tsv.gz"
path3="data/title.ratings.tsv.gz"

In [82]:
gdp_rank = gdpRank()

In [83]:
rank = fullTask(path1,
                path2,
                path3,
                Ks = [1000],
                Ns = range(10, 201, 10),
                verbose = True,
                file = "answer.txt")

Loading data... 

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


Done!
Preprocessing... Done!
Creating ranking...

Done!


In [87]:
df, df3 = load_dataframes(path1, path2, path3)
df_analysis = prepareDataset(df, df3)

In [ ]:
df_analysis.head()